In [12]:
import os
import json
import requests

from googleapiclient import discovery
from google.oauth2 import service_account


In [28]:
def fetch_cg_info(date):
    response = requests.get(f'https://api.coingecko.com/api/v3/coins/olympus/history?date={date}&localization=false')
    r = response.json()
    price = r['market_data']['current_price']['usd']
    mcap = r['market_data']['market_cap']['usd']
    return [price, int(mcap/price)]


def fetch_value(spreadsheet_id, sheet, cell_range):
    result = sheet.values().get(spreadsheetId=spreadsheet_id, range=cell_range).execute()
    return result.get('values',[])[0][0]

In [32]:
scopes = ["https://www.googleapis.com/auth/spreadsheets"]
secret_file = os.path.join(os.getcwd(), 'client_secret.json')
spreadsheet_id = "1cTYr0Kimk2O29t136BaKaePqqMEsb4xTojdTsMBZf3A"

try:    
    credentials = service_account.Credentials.from_service_account_file(secret_file, scopes=scopes)
    service = discovery.build('sheets', 'v4', credentials=credentials)
    sheet = service.spreadsheets()

    for row in range (2, 126):
        date = fetch_value(spreadsheet_id, sheet, cell_range=f'OHM!C{row}:C{row}')
        values = fetch_cg_info(date)
        info = {'values' : [values]}
        sheet.values().update(spreadsheetId=spreadsheet_id, body=info, range=f'OHM!D{row}:E{row}', valueInputOption='USER_ENTERED').execute()

except OSError as e:
    print (e)